In [10]:
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple, defaultdict
import heapq
import math
from dataclasses import dataclass, field
from typing import List, Set, Dict, Tuple, Optional

# ===== Data Structures =====
# Point class to represent sites and endpoints
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f"Point({self.x:.2f}, {self.y:.2f})"
    
    def distance_to(self, other):
        return math.sqrt((self.x - other.x)**2 + (self.y - other.y)**2)

# Edge class to represent a Voronoi edge
class Edge:
    def __init__(self, start=None, end=None, site1=None, site2=None):
        self.start = start  # start vertex
        self.end = end      # end vertex
        self.site1 = site1  # one of the sites defining this edge
        self.site2 = site2  # the other site
        
        # Direction vector for the edge, perpendicular to the line connecting the sites
        if site1 and site2:
            dx = site2.x - site1.x
            dy = site2.y - site1.y
            # Perpendicular direction vector
            self.direction = Point(-dy, dx)
            # Normalize
            length = math.sqrt(self.direction.x**2 + self.direction.y**2)
            if length > 0:
                self.direction.x /= length
                self.direction.y /= length
        else:
            self.direction = None
    
    def __repr__(self):
        return f"Edge({self.start}, {self.end})"

# Event types for the sweep line algorithm
class Event:
    SITE = 0
    CIRCLE = 1

@dataclass(order=True)
class SiteEvent:
    x: float
    point: Point = field(compare=False)
    type: int = field(default=Event.SITE, compare=False)

@dataclass(order=True)
class CircleEvent:
    x: float  # x-coordinate of the circle event point
    y: float = field(compare=False)  # y-coordinate
    arc: 'Arc' = field(compare=False)  # arc that will disappear
    vertex: Point = field(compare=False)  # The Voronoi vertex being created
    type: int = field(default=Event.CIRCLE, compare=False)

# Arc class to represent beach line arcs
class Arc:
    def __init__(self, site, prev=None, next=None):
        self.site = site        # Site that generates this arc
        self.prev = prev        # Previous arc in the beach line
        self.next = next        # Next arc in the beach line
        self.event = None       # Circle event where this arc disappears
        self.left_edge = None   # Edge to the left of this arc
        self.right_edge = None  # Edge to the right of this arc

# ===== Helper Functions =====
def find_breakpoint(p1, p2, directrix):
    """
    Find the breakpoint between two parabolas defined by sites p1, p2 and a directrix.
    """
    # Safety check for directrix - it must be strictly below both p1.y and p2.y
    if directrix >= p1.y or directrix >= p2.y:
        # In this case, we can't form proper parabolas - we're at the same level as a site
        if p1.y > p2.y:
            return Point(p2.x, float('inf'))  # p2 is higher, use its x-coordinate
        else:
            return Point(p1.x, float('inf'))  # p1 is higher, use its x-coordinate
    
    # Handle special cases first
    if abs(p1.y - p2.y) < 1e-10:  # Sites have the same y-coordinate
        return Point((p1.x + p2.x) / 2, float('inf'))
    
    # If the sites have the same x-coordinate, the breakpoint is on that vertical line
    if abs(p1.x - p2.x) < 1e-10:
        x = p1.x
        # Calculate y coordinate of parabola at this x
        y = ((x - p1.x)**2) / (2 * (p1.y - directrix)) + (p1.y + directrix) / 2
        return Point(x, y)
    
    try:
        # General case: solve the quadratic equation to find the x-coordinate of the breakpoint
        # The equation is derived from equating the two parabola equations
        h1, k1 = p1.x, p1.y
        h2, k2 = p2.x, p2.y
        
        # Make sure the denominators aren't zero
        if abs(k1 - directrix) < 1e-10 or abs(k2 - directrix) < 1e-10:
            # If one of the denominators is near zero, that site is close to the sweep line
            # Use the x-coordinate of the other site as

def calculate_circle_event(arc, sweep_line):
    """
    Calculate a potential circle event for the given arc.
    Returns a CircleEvent if one exists, None otherwise.
    """
    # An arc can only have a circle event if it has both a previous and next arc
    if not arc.prev or not arc.next:
        return None
    
    # The sites that define the three consecutive arcs
    a = arc.prev.site
    b = arc.site
    c = arc.next.site
    
    # Check if any two sites are the same (can happen due to precision issues)
    if (abs(a.x - b.x) < 1e-10 and abs(a.y - b.y) < 1e-10) or \
       (abs(b.x - c.x) < 1e-10 and abs(b.y - c.y) < 1e-10) or \
       (abs(a.x - c.x) < 1e-10 and abs(a.y - c.y) < 1e-10):
        return None
    
    # Check if the points are nearly collinear (which means no circle event)
    # We do this by checking if the determinant is close to zero
    det = (b.x - a.x) * (c.y - a.y) - (c.x - a.x) * (b.y - a.y)
    if abs(det) < 1e-10:
        return None  # Points are collinear, no circle event
    
    # Check if the points make a counterclockwise turn
    # In a Voronoi diagram, we're only interested in circle events where 
    # the three sites make a counterclockwise turn
    if det <= 0:
        return None
    
    # Calculate the center of the circle through the three points
    # Using the perpendicular bisector method
    
    # Perpendicular bisector of AB
    mx1 = (a.x + b.x) / 2
    my1 = (a.y + b.y) / 2
    dx1 = b.x - a.x
    dy1 = b.y - a.y
    
    # Perpendicular bisector of BC
    mx2 = (b.x + c.x) / 2
    my2 = (b.y + c.y) / 2
    dx2 = c.x - b.x
    dy2 = c.y - b.y
    
    # Intersection of the two perpendicular bisectors is the circle center
    # We're solving the system of equations for lines through (mx1,my1) and (mx2,my2)
    # with direction vectors (-dy1,dx1) and (-dy2,dx2)
    
    # Handle parallel cases
    if abs(dx1 * dy2 - dy1 * dx2) < 1e-10:
        return None  # Parallel bisectors, no unique circle
    
    # Parametric solution to find the intersection
    try:
        t = ((mx2 - mx1) * dy2 - (my2 - my1) * dx2) / (dx1 * dy2 - dy1 * dx2)
    except ZeroDivisionError:
        return None
    
    # Circle center
    center_x = mx1 - t * dy1
    center_y = my1 + t * dx1
    center = Point(center_x, center_y)
    
    # Circle radius
    radius = center.distance_to(a)
    
    # Lowest point of the circle is the y-coordinate minus the radius
    lowest_point_y = center_y - radius
    
    # If the lowest point is above the sweep line, this circle event hasn't happened yet
    # But we still want to schedule it for the future
    if lowest_point_y <= sweep_line:
        return None  # This circle event is in the past, ignore it
    
    # The x-coordinate of the event is the x-coordinate of the center
    x = center_x
    
    # Create and return the circle event
    return CircleEvent(x=x, y=lowest_point_y, arc=arc, vertex=center)

def bisect_sites(site1, site2):
    """
    Calculate the perpendicular bisector line between two sites.
    Returns two points on the bisector.
    """
    # Midpoint between the sites
    mid_x = (site1.x + site2.x) / 2
    mid_y = (site1.y + site2.y) / 2
    
    # Direction vector from site1 to site2
    dx = site2.x - site1.x
    dy = site2.y - site1.y
    
    # Perpendicular direction vector (rotate 90 degrees)
    perp_dx = -dy
    perp_dy = dx
    
    # Normalize the perpendicular vector
    length = math.sqrt(perp_dx**2 + perp_dy**2)
    if length > 0:  # Avoid division by zero
        perp_dx /= length
        perp_dy /= length
    
    # Two points on the bisector
    p1 = Point(mid_x + perp_dx * 100, mid_y + perp_dy * 100)
    p2 = Point(mid_x - perp_dx * 100, mid_y - perp_dy * 100)
    
    return p1, p2

def find_arc_above(beach_line, x, sweep_line):
    """
    Find the arc in the beach line that is directly above the given x-coordinate.
    """
    if not beach_line:
        return None
    
    current = beach_line
    
    # Special case: if there's only one arc in the beach line
    if not current.next:
        return current
    
    # March through the beach line to find the arc above x
    while current:
        # If we're at the last arc, this must be the one
        if not current.next:
            return current
        
        # Calculate the breakpoint between current and next
        try:
            breakpoint = find_breakpoint(current.site, current.next.site, sweep_line)
            
            # If breakpoint calculation failed or x is to the left of the breakpoint,
            # we're in the current arc
            if not breakpoint or x < breakpoint.x:
                return current
            
        except Exception:
            # If there's an error calculating the breakpoint, just use this arc
            # (This can happen due to numerical issues)
            return current
        
        # Otherwise, move to the next arc
        current = current.next
    
    # If we somehow didn't find an arc (shouldn't happen), return the first arc
    return beach_line

# ===== Main Algorithm =====
def fortunes_algorithm(sites, width, height):
    """
    Implement Fortune's algorithm to construct a Voronoi diagram.
    Returns edges and vertices of the diagram.
    """
    # Initialize the event queue with site events
    event_queue = []
    for site in sites:
        heapq.heappush(event_queue, SiteEvent(x=site.x, point=site))
    
    # Initialize data structures
    beach_line = None  # The beach line (a doubly-linked list of Arc objects)
    edges = []         # List of Voronoi edges
    vertices = []      # List of Voronoi vertices
    
    # Main loop
    sweep_line = 0
    while event_queue:
        event = heapq.heappop(event_queue)
        
        # Update the sweep line position
        if event.type == Event.SITE:
            sweep_line = event.point.y
        else:  # Circle event
            sweep_line = event.y
        
        # Handle site event
        if event.type == Event.SITE:
            beach_line = handle_site_event(event, beach_line, event_queue, edges, sweep_line)
        # Handle circle event
        else:
            beach_line = handle_circle_event(event, beach_line, event_queue, edges, vertices, sweep_line)
    
    # Complete and clean up the diagram
    complete_edges(edges, width, height)
    
    return edges, vertices

def handle_site_event(event, beach_line, event_queue, edges, sweep_line):
    """
    Handle a site event in Fortune's algorithm.
    """
    site = event.point
    
    # If the beach line is empty, add the first arc and return
    if not beach_line:
        return Arc(site)
    
    # Find the arc directly above the new site
    arc_above = find_arc_above(beach_line, site.x, sweep_line)
    if not arc_above:
        return beach_line  # Something went wrong, just return current beach line
    
    # If there's an associated circle event, remove it
    if arc_above.event:
        try:
            event_queue.remove(arc_above.event)
        except ValueError:
            pass  # Event might already be processed
        arc_above.event = None
    
    # Create a new edge between the site of the arc above and the new site
    new_edge = Edge(site1=arc_above.site, site2=site)
    edges.append(new_edge)
    
    # Split the arc above into three parts: left, middle (with the new site), and right
    left_arc = Arc(arc_above.site)
    new_arc = Arc(site)
    right_arc = Arc(arc_above.site)
    
    # Update the beach line structure
    left_arc.next = new_arc
    new_arc.prev = left_arc
    new_arc.next = right_arc
    right_arc.prev = new_arc
    
    # Connect with the rest of the beach line
    if arc_above.prev:
        arc_above.prev.next = left_arc
        left_arc.prev = arc_above.prev
    else:
        beach_line = left_arc  # This becomes the new start of the beach line
    
    if arc_above.next:
        right_arc.next = arc_above.next
        arc_above.next.prev = right_arc
    
    # Set edge references
    left_arc.right_edge = new_edge
    new_arc.left_edge = new_edge
    
    # Create another edge for the right side
    right_edge = Edge(site1=site, site2=arc_above.site)
    edges.append(right_edge)
    new_arc.right_edge = right_edge
    right_arc.left_edge = right_edge
    
    # Check for new circle events
    check_circle_event(left_arc, event_queue, sweep_line)
    check_circle_event(right_arc, event_queue, sweep_line)
    
    return beach_line

def handle_circle_event(event, beach_line, event_queue, edges, vertices, sweep_line):
    """
    Handle a circle event in Fortune's algorithm.
    """
    arc = event.arc
    
    # Store the Voronoi vertex (center of the circle)
    vertex = event.vertex
    vertices.append(vertex)
    
    # Get the arcs to the left and right
    left_arc = arc.prev
    right_arc = arc.next
    
    # If these arcs don't exist, something went wrong
    if not left_arc or not right_arc:
        return beach_line
    
    # If there are circle events associated with these arcs, remove them
    if left_arc.event:
        try:
            event_queue.remove(left_arc.event)
        except ValueError:
            pass
        left_arc.event = None
    
    if right_arc.event:
        try:
            event_queue.remove(right_arc.event)
        except ValueError:
            pass
        right_arc.event = None
    
    # Create a new edge between the sites of the left and right arcs
    new_edge = Edge(start=vertex, site1=left_arc.site, site2=right_arc.site)
    edges.append(new_edge)
    
    # Set the end point of the edges that meet at this vertex
    if arc.left_edge:
        arc.left_edge.end = vertex
    if arc.right_edge:
        arc.right_edge.end = vertex
    
    # Remove the arc from the beach line
    if left_arc:
        left_arc.next = right_arc
    if right_arc:
        right_arc.prev = left_arc
    
    # If the arc was the first one in the beach line, update the beach line pointer
    if beach_line == arc:
        beach_line = right_arc
    
    # Update edge references
    if left_arc and right_arc:
        left_arc.right_edge = new_edge
        right_arc.left_edge = new_edge
    
    # Check for new circle events
    if left_arc:
        check_circle_event(left_arc, event_queue, sweep_line)
    if right_arc:
        check_circle_event(right_arc, event_queue, sweep_line)
    
    return beach_line

def check_circle_event(arc, event_queue, sweep_line):
    """
    Check if a new circle event needs to be added for the given arc.
    """
    if not arc or not arc.prev or not arc.next:
        return
    
    circle_event = calculate_circle_event(arc, sweep_line)
    if circle_event:
        arc.event = circle_event
        heapq.heappush(event_queue, circle_event)

def complete_edges(edges, width, height):
    """
    Complete any unfinished edges by extending them to the boundaries.
    """
    for edge in edges:
        # Skip edges that already have both endpoints
        if edge.start and edge.end:
            continue
        
        # Get the direction of the edge (perpendicular to the line connecting the sites)
        if not edge.site1 or not edge.site2:
            continue  # Skip if sites are missing
        
        dx = edge.site2.x - edge.site1.x
        dy = edge.site2.y - edge.site1.y
        
        # Midpoint between the sites
        mx = (edge.site1.x + edge.site2.x) / 2
        my = (edge.site1.y + edge.site2.y) / 2
        
        # Perpendicular direction
        perp_dx = -dy
        perp_dy = dx
        
        # Normalize
        length = math.sqrt(perp_dx**2 + perp_dy**2)
        if length > 0:
            perp_dx /= length
            perp_dy /= length
        
        # Extend in both directions from the start point or midpoint
        start_point = edge.start if edge.start else Point(mx, my)
        
        # Calculate intersection with boundaries
        # We need to find t such that start_point + t*direction is on the boundary
        
        # Try all four boundaries
        ts = []
        
        # Left boundary (x = 0)
        if abs(perp_dx) > 1e-10:
            t = (0 - start_point.x) / perp_dx
            y = start_point.y + t * perp_dy
            if 0 <= y <= height:
                ts.append((t, Point(0, y)))
        
        # Right boundary (x = width)
        if abs(perp_dx) > 1e-10:
            t = (width - start_point.x) / perp_dx
            y = start_point.y + t * perp_dy
            if 0 <= y <= height:
                ts.append((t, Point(width, y)))
        
        # Bottom boundary (y = 0)
        if abs(perp_dy) > 1e-10:
            t = (0 - start_point.y) / perp_dy
            x = start_point.x + t * perp_dx
            if 0 <= x <= width:
                ts.append((t, Point(x, 0)))
        
        # Top boundary (y = height)
        if abs(perp_dy) > 1e-10:
            t = (height - start_point.y) / perp_dy
            x = start_point.x + t * perp_dx
            if 0 <= x <= width:
                ts.append((t, Point(x, height)))
        
        # Sort by parameter t
        ts.sort()
        
        # We need the two intersections with t values of opposite signs
        # or the two smallest positive or largest negative if we don't cross the midpoint
        if not ts:
            continue  # No intersections, skip this edge
        
        if len(ts) == 1:
            if not edge.start:
                edge.start = ts[0][1]
            elif not edge.end:
                edge.end = ts[0][1]
            continue
        
        # Find intersections on both sides of the midpoint
        negative_ts = [(t, p) for t, p in ts if t < 0]
        positive_ts = [(t, p) for t, p in ts if t >= 0]
        
        if negative_ts and positive_ts:
            # Take the closest negative and closest positive
            if not edge.start:
                edge.start = negative_ts[-1][1]  # largest negative t
            if not edge.end:
                edge.end = positive_ts[0][1]    # smallest positive t
        elif negative_ts:
            # Take the two largest negative
            if len(negative_ts) >= 2:
                if not edge.start:
                    edge.start = negative_ts[-2][1]  # second largest negative t
                if not edge.end:
                    edge.end = negative_ts[-1][1]    # largest negative t
        elif positive_ts:
            # Take the two smallest positive
            if len(positive_ts) >= 2:
                if not edge.start:
                    edge.start = positive_ts[0][1]  # smallest positive t
                if not edge.end:
                    edge.end = positive_ts[1][1]    # second smallest positive t

def generate_random_points(n, width, height, max_distance=None):
    """
    Generate n random points within a given width and height,
    ensuring they're not too far from each other.
    """
    if n <= 0:
        return []
    
    # Generate the first point randomly
    points = [Point(np.random.uniform(0.2*width, 0.8*width), 
                   np.random.uniform(0.2*height, 0.8*height))]
    
    # If max_distance is not specified, set it to about 1/3 of the diagonal
    if max_distance is None:
        max_distance = math.sqrt(width**2 + height**2) / 3
    
    # Generate remaining points
    while len(points) < n:
        # Pick a random existing point
        reference_point = points[np.random.randint(0, len(points))]
        
        # Generate a point at a random angle and random distance from the reference
        angle = np.random.uniform(0, 2 * math.pi)
        distance = np.random.uniform(0, max_distance)
        
        x = reference_point.x + distance * math.cos(angle)
        y = reference_point.y + distance * math.sin(angle)
        
        # Ensure the point is within bounds
        x = max(0, min(width, x))
        y = max(0, min(height, y))
        
        points.append(Point(x, y))
    
    return points

def plot_voronoi(sites, edges, width, height):
    """
    Plot the Voronoi diagram.
    """
    plt.figure(figsize=(10, 10))
    
    # Plot edges
    for edge in edges:
        if edge.start and edge.end:
            plt.plot([edge.start.x, edge.end.x], [edge.start.y, edge.end.y], 'b-')
    
    # Plot sites
    site_xs = [site.x for site in sites]
    site_ys = [site.y for site in sites]
    plt.scatter(site_xs, site_ys, c='r', s=50)
    
    # Add labels to sites
    for i, site in enumerate(sites):
        plt.text(site.x + 0.1, site.y + 0.1, f'{i}', fontsize=12)
    
    plt.xlim(0, width)
    plt.ylim(0, height)
    plt.grid(True)
    plt.title("Voronoi Diagram using Fortune's Algorithm")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.gca().set_aspect('equal')
    plt.show()

# ===== Run the algorithm =====
def main():
    # Set the bounds
    width, height = 10, 10
    
    # Generate random sites
    np.random.seed(42)  # For reproducibility
    num_sites = 10
    
    # Option 1: Generate points not too far from each other
    sites = generate_random_points(num_sites, width, height, max_distance=3)
    
    # Option 2: Generate points with small jitter to avoid having identical points
    # This helps in case your generated points have duplicates
    # (noticed that there are duplicated points in the error message)
    unique_sites = []
    for site in sites:
        # Check if this is very close to an existing site
        is_duplicate = False
        for existing in unique_sites:
            if site.distance_to(existing) < 0.1:  # Threshold for considering as duplicate
                is_duplicate = True
                break
        
        if not is_duplicate:
            unique_sites.append(site)
        else:
            # Add a slightly jittered version
            jittered = Point(site.x + np.random.uniform(-0.2, 0.2), 
                            site.y + np.random.uniform(-0.2, 0.2))
            unique_sites.append(jittered)
    
    sites = unique_sites
    
    print(f"Generated {len(sites)} sites:")
    for i, site in enumerate(sites):
        print(f"Site {i}: ({site.x:.2f}, {site.y:.2f})")
    
    # Run Fortune's algorithm
    edges, vertices = fortunes_algorithm(sites, width, height)
    
    print(f"\nGenerated {len(edges)} edges and {len(vertices)} vertices.")
    
    # Plot the result
    plot_voronoi(sites, edges, width, height)

if __name__ == "__main__":
    main()

IndentationError: expected an indented block after 'if' statement on line 107 (1411455025.py, line 111)